In [1]:
#imports
import pandas as pd
from PIL import Image
import io
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
import datetime, os
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
#import data
X = np.load("../data/X.npy")
y = np.load("../data/y.npy")

In [5]:
#split into training and test sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=True, random_state=42
)

In [7]:
#convert to dataset for tensorflow
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

train_dataset = train_dataset.shuffle(5000).batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

In [9]:
#unbatch data
train_unbatched = train_dataset.unbatch()

# Create a validation split of 10%
val_frac = 0.1
train_size = int((1 - val_frac) * len(X_train)) 

train_ds = train_unbatched.take(train_size).batch(32).prefetch(tf.data.AUTOTUNE)
val_ds   = train_unbatched.skip(train_size).batch(32).prefetch(tf.data.AUTOTUNE)


In [ ]:
#save test dataset
np.savez_compressed(
    "../data/test_dataset.npz",
    X_test=X_test,
    y_test=y_test
)

In [13]:
#model structure
IMG_SIZE = 64   #resize images to be square, can change size

def make_simple_cnn(input_shape=(IMG_SIZE, IMG_SIZE, 1), num_classes=26):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv2D(32, (3,3), activation='relu', padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(2)(x)

    x = layers.Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(2)(x)

    x = layers.Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(2)(x)

    x = layers.Flatten()(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.4)(x)

    outputs = layers.Dense(num_classes, activation='softmax')(x)
    model = models.Model(inputs, outputs)
    return model

model = make_simple_cnn()
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 64, 64, 1)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 64, 64, 32)          │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 64, 64, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 32, 32, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 32, 32, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 32, 32, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 16, 16, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 16, 16, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 16, 16, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 8, 8, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 8192)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 8192)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │       2,097,408 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 256)                 │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 26)                  │           6,682 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,198,682 (8.39 MB)

 Trainable params: 2,197,722 (8.38 MB)

 Non-trainable params: 960 (3.75 KB)

In [15]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss='categorical_crossentropy',   # your labels are one-hot
    metrics=['accuracy']
)

In [17]:
#model fit settings
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

callbacks = [
    ModelCheckpoint("best_model.h5", monitor="val_accuracy", save_best_only=True, verbose=1),
    EarlyStopping(monitor="val_loss", patience=8, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, min_lr=1e-6, verbose=1),
    TensorBoard(log_dir=logdir)
]

In [19]:
epochs = 50
#fit model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    callbacks=callbacks
)

Epoch 1/30
    245/Unknown 53s 180ms/step - accuracy: 0.0503 - loss: 3.9258

C:\Users\Chloe\anaconda3\Lib\site-packages\keras\src\trainers\epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()



Epoch 1: val_accuracy improved from -inf to 0.04598, saving model to best_model.h5


245/245 ━━━━━━━━━━━━━━━━━━━━ 56s 189ms/step - accuracy: 0.0503 - loss: 3.9250 - val_accuracy: 0.0460 - val_loss: 3.9526 - learning_rate: 0.0010
Epoch 2/30
245/245 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step - accuracy: 0.1081 - loss: 3.3208
Epoch 2: val_accuracy improved from 0.04598 to 0.09080, saving model to best_model.h5


245/245 ━━━━━━━━━━━━━━━━━━━━ 79s 181ms/step - accuracy: 0.1081 - loss: 3.3207 - val_accuracy: 0.0908 - val_loss: 3.3003 - learning_rate: 0.0010
Epoch 3/30
245/245 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step - accuracy: 0.1529 - loss: 3.0444
Epoch 3: val_accuracy improved from 0.09080 to 0.20805, saving model to best_model.h5


...

245/245 ━━━━━━━━━━━━━━━━━━━━ 45s 185ms/step - accuracy: 0.7741 - loss: 0.7530 - val_accuracy: 0.9529 - val_loss: 0.2850 - learning_rate: 0.0010
Epoch 14/30
245/245 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - accuracy: 0.7985 - loss: 0.6664
Epoch 14: val_accuracy did not improve from 0.95287
245/245 ━━━━━━━━━━━━━━━━━━━━ 29s 116ms/step - accuracy: 0.7985 - loss: 0.6665 - val_accuracy: 0.9195 - val_loss: 0.3849 - learning_rate: 0.0010
Epoch 15/30
245/245 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - accuracy: 0.8049 - loss: 0.6295
Epoch 15: val_accuracy improved from 0.95287 to 0.98046, saving model to best_model.h5


245/245 ━━━━━━━━━━━━━━━━━━━━ 28s 114ms/step - accuracy: 0.8049 - loss: 0.6296 - val_accuracy: 0.9805 - val_loss: 0.1534 - learning_rate: 0.0010
Epoch 16/30
245/245 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step - accuracy: 0.8242 - loss: 0.5410
Epoch 16: val_accuracy did not improve from 0.98046
245/245 ━━━━━━━━━━━━━━━━━━━━ 28s 112ms/step - accuracy: 0.8242 - loss: 0.5411 - val_accuracy: 0.8885 - val_loss: 0.4067 - learning_rate: 0.0010
Epoch 17/30
245/245 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - accuracy: 0.8520 - loss: 0.4755
Epoch 17: val_accuracy did not improve from 0.98046
245/245 ━━━━━━━━━━━━━━━━━━━━ 28s 116ms/step - accuracy: 0.8519 - loss: 0.4756 - val_accuracy: 0.9759 - val_loss: 0.1570 - learning_rate: 0.0010
Epoch 18/30
245/245 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - accuracy: 0.8466 - loss: 0.4619
Epoch 18: val_accuracy did not improve from 0.98046

Epoch 18: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
245/245 ━━━━━━━━━━━━━━━━━━━━ 30s 122ms/step - accuracy: 0.8465 - lo

245/245 ━━━━━━━━━━━━━━━━━━━━ 31s 125ms/step - accuracy: 0.8898 - loss: 0.3603 - val_accuracy: 0.9977 - val_loss: 0.0446 - learning_rate: 5.0000e-04
Epoch 20/30
245/245 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - accuracy: 0.9103 - loss: 0.2905
Epoch 20: val_accuracy did not improve from 0.99770
245/245 ━━━━━━━━━━━━━━━━━━━━ 29s 119ms/step - accuracy: 0.9103 - loss: 0.2905 - val_accuracy: 0.9954 - val_loss: 0.0423 - learning_rate: 5.0000e-04
Epoch 21/30
245/245 ━━━━━━━━━━━━━━━━━━━━ 0s 119ms/step - accuracy: 0.9265 - loss: 0.2484
Epoch 21: val_accuracy improved from 0.99770 to 0.99885, saving model to best_model.h5


245/245 ━━━━━━━━━━━━━━━━━━━━ 30s 123ms/step - accuracy: 0.9265 - loss: 0.2484 - val_accuracy: 0.9989 - val_loss: 0.0321 - learning_rate: 5.0000e-04
Epoch 22/30
245/245 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step - accuracy: 0.9254 - loss: 0.2487
Epoch 22: val_accuracy improved from 0.99885 to 1.00000, saving model to best_model.h5


245/245 ━━━━━━━━━━━━━━━━━━━━ 30s 123ms/step - accuracy: 0.9254 - loss: 0.2487 - val_accuracy: 1.0000 - val_loss: 0.0158 - learning_rate: 5.0000e-04
Epoch 23/30
245/245 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 0.9316 - loss: 0.2235
Epoch 23: val_accuracy did not improve from 1.00000
245/245 ━━━━━━━━━━━━━━━━━━━━ 31s 127ms/step - accuracy: 0.9316 - loss: 0.2235 - val_accuracy: 0.9989 - val_loss: 0.0140 - learning_rate: 5.0000e-04
Epoch 24/30
245/245 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 0.9384 - loss: 0.2085
Epoch 24: val_accuracy did not improve from 1.00000
245/245 ━━━━━━━━━━━━━━━━━━━━ 31s 127ms/step - accuracy: 0.9384 - loss: 0.2086 - val_accuracy: 0.9966 - val_loss: 0.0375 - learning_rate: 5.0000e-04
Epoch 25/30
245/245 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step - accuracy: 0.9431 - loss: 0.1913
Epoch 25: val_accuracy did not improve from 1.00000
245/245 ━━━━━━━━━━━━━━━━━━━━ 30s 123ms/step - accuracy: 0.9431 - loss: 0.1914 - val_accuracy: 1.0000 - val_loss: 0.0202 - learning_ra

In [ ]:
#save the model
model.save("../models/asl_model_50epochs.keras")